# Setup
Utilizaremos a biblioteca `boto3` para nos comunicarmos com a AWS.

Utilizaremos as bibliotecas `pandas` e `pyspark` para lidarmos com nossos dados.

Utilizaremos as bibliotecas `matplotlib` e `seaborn` para a análise final.

In [11]:
import pandas as pd
import boto3

import findspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [12]:
BUCKET = '896309849144'
LAYERS_PREFIX = f'layers'# f's3://{BUCKET}/layers'
GOLD = f'{LAYERS_PREFIX}/gold'
SILVER = f'{LAYERS_PREFIX}/silver'
BRONZE = f'{LAYERS_PREFIX}/bronze'

LOCAL_BRONZE = './local/bronze'
LOCAL_SILVER = './local/silver'


In [13]:
s3_client = boto3.client('s3')

def upload_to_s3(
        file_path : str,
        bucket : str,
        bucket_path : str):
    '''
    Uploads file_path to the given bucket, as bucket_path (include the name of the file in bucket_path)
    
    Parameters
    -
    file_path : str
    bucket : str
    bucket_path : str
    '''
    s3_client.upload_file(file_path, bucket, bucket_path)

# 0. Ingestion
Faremos o upload dos dados brutos extraídos manualmente do IBGE, a um bucket AWS S3 via a biblioteca `boto3`.

Conforme o pedido, utilizaremos os últimos 3 meses disponíveis.

In [8]:
files = [
    f'{LOCAL_BRONZE}/PNAD_COVID_092020.csv',
    f'{LOCAL_BRONZE}/PNAD_COVID_102020.csv',
    f'{LOCAL_BRONZE}/PNAD_COVID_112020.csv']

In [9]:
for file in files:
    print(file.split('/')[-1])

PNAD_COVID_092020.csv
PNAD_COVID_102020.csv
PNAD_COVID_112020.csv


In [10]:
for file in files:
    upload_to_s3(
        file,
        BUCKET,
        f'{BRONZE}/{file.split("/")[-1]}')

S3UploadFailedError: Failed to upload ./local/bronze/PNAD_COVID_092020.csv to 896309849144/layers/bronze/PNAD_COVID_092020.csv: An error occurred (InvalidToken) when calling the CreateMultipartUpload operation: The provided token is malformed or otherwise invalid.

# 1. Bronze
O processo de re-estruturamento dos dados é feito na AWS via `AWS Glue`.

# 2. Silver
Pegaremos esses dados re-estruturados *silver* e faremos nossas queries via `PySpark`.

In [14]:
to_download = s3_client.list_objects_v2(
    Bucket = BUCKET,
    Prefix = SILVER
)['Contents'][1:]

count = 1

for file in to_download:
    s3_client.download_file(BUCKET, file['Key'], f'{LOCAL_SILVER}/silver_{count}.parquet')
    count+=1

ClientError: An error occurred (InvalidToken) when calling the ListObjectsV2 operation: The provided token is malformed or otherwise invalid.

## Checando integridade dos dados

Comparemos o tamanho da camade bronze original com a camada silver atual para nos certificarmos que o processo de merge ocorreu corretamente

In [ ]:
df_silver = pd.read_parquet('./local/')
df_silver.shape[0]

1149197

In [ ]:
row_count = 0
for f in files[:]:
    _ = pd.read_csv(f)
    row_count += _.shape[0]

    print(_.shape[0])
print(f'TOTAL = {row_count}')

387298
380461
381438
TOTAL = 1149197


## Selecionamento das perguntas
Foi requesitado a utilização de no máximo 20 perguntas da pesquisa. Seguem as selecionadas abaixo.
- `UF` Unidade da Federação
 - `V1022` Situação de Domicílio
 - `V1030` Projeção da população 
 - `A002` Qual sua idade?
 - `A004` Cor ou raça:
 - `A005` Qual sua escolaridade?
 - `B002` Por causa (de sintomas), foi a algum estabelecimento de saúde?
 - `B007` O(A) Sr(a) tem algum plano de saúde médico, seja particular, de empresa ou de órgão público?
 - `B008` O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus?
 - `B009B` (SWAB) Qual o resultado?
 - `B009D` (Furo no dedo) Qual o resultado?
 - `B009F` (Veia no braço) Qual o resultado?
 - `B010` Algum médico já lhe deu o diagnóstico de alguma dessas doenças?
 - `B011` Na semana passada, devido à pandemia do Coronavírus, em que medida o(a) Sr(a) restringiu o contato com as pessoas?
 - `C001` Na semana passada, por pelo menos uma hora, o(a) Sr(a) trabalhou ou fez algum bico?
 - `C003` Qual o principal motivo deste afastamento temporário?
 - `C007` No trabalho (único ou principal) que tinha nessa semana, o(a) Sr(a) era:
 - `C010` Número de faixa do rendimento / retirada em dinheiro (NORMALMENTE)
 - `C011` Número de faixa do rendimento / retirada em dinheiro (NO MES ATUAL)
 - `C012` Na maior parte do tempo, na semana passada, esse trabalho (único ou principal) foi exercido no mesmo local em que costuma trabalhar?

## PySpark

In [15]:
findspark.init()

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName('PySpark Dataframe').getOrCreate()

In [16]:
import os
print(os.environ['HADOOP_HOME']) 
print(os.environ['JAVA_HOME']) 

C:\spark_setup\spark-3.5.6-bin-hadoop3\hadoop
C:\Users\Victoria\anaconda3\envs\venv\Library\lib\jvm


In [17]:
df_silver = spark.read.parquet(LOCAL_SILVER)